## Visualizing QFT

In [ ]:
!pip install qiskit
!pip install pylatexenc
!pip install imageio

In [2]:
import imageio
import numpy as np
from qiskit import QuantumCircuit, assemble, Aer, execute
from math import pi, sqrt, sin,cos
from qiskit.visualization import plot_bloch_multivector, plot_histogram
sim = Aer.get_backend('aer_simulator')

In [3]:
# Taken from Qiskit tutorial
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [5]:


def visualize_qft(n_qubits,init_func,steps=10, fps=5,figname='states.gif'):
    n = n_qubits
    
    init_states = list(map(init_func,[i/(steps-1) for i in range(steps) ]))

    for s_i, init_state in enumerate(init_states):

        for p in range(2):
            qc = QuantumCircuit(n,n)

            for i in range(len(init_state)):
                #qc.ry(pi*0.5,i)# TODO
                #qc.rz(init_state[i],i)
                qc.rx(init_state[i],i)

            if p == 0:
                # Save encoded input
                qc.save_statevector()
                qobj = assemble(qc)

                state = sim.run(qobj).result().get_statevector()
                plot_bloch_multivector(state,filename = f'state_input_{s_i}.png')
                continue

            # Save QFT output
            qc = qft(qc,n)

            qc.save_statevector()
            qobj = assemble(qc)

            state = sim.run(qobj).result().get_statevector()
            plot_bloch_multivector(state,filename = f'state_{s_i}.png')

    images = []
    for filename_input,filename_output in [ (f'state_input_{s_i}.png',f'state_{s_i}.png') for s_i in range(len(init_states))]:
        im_in = imageio.imread(filename_input)
        im_out = imageio.imread(filename_output)
                                             
        im = np.concatenate([im_in,im_out])
        images.append(im)
    imageio.mimsave(figname, images, duration=1./fps)

    print('DONE. Saved {}'.format(figname))



## Visualize specific cases

In [6]:
# Binary encoding
n = 4
steps = 2**n

def sweep_bitencode(p):
    s = []
    i = int(p*(steps-1))
    for d in range(n):
        s.append( pi*(i//(2**d) % 2))
    
    return s


visualize_qft(n_qubits=n,init_func=sweep_bitencode,steps=steps,fps=1,figname='states_bitencode.gif')

DONE. Saved states_bitencode.gif


<img src="states_bitencode.gif" width="100%" align="center">

In [7]:
# Sweep qubit0 rotation
n = 4
def sweep_qubit0(p):
    s = [0.0 for _ in range(n)]
    s[0] = pi*p
    return s


visualize_qft(n_qubits=n,init_func=sweep_qubit0)

DONE. Saved states.gif



<img src="states.gif" width="100%" align="center">